In [21]:
import arcpy
import sys
import pandas as pd
sys.path.append(r'D:\PythonFiles\ArcGisModule')
import function

In [47]:
filename = r'C:\Users\Fei\Documents\ArcGIS\Projects\Cleaned Classification\Lawrence.gdb\map20211219093000'
path_gdb = r'C:\Users\Fei\Documents\ArcGIS\Projects\Cleaned Classification\Lawrence.gdb'
path_temp = r"C:\Users\Fei\Documents\ArcGIS\Projects\Cleaned Classification\Output"
wildcard = 'map20211219*'

In [23]:
def ReadFeatureLayer(filename, path_gdb, path_temp):
    with arcpy.EnvManager(scratchWorkspace=path_gdb, workspace=path_gdb):
        with arcpy.EnvManager(qualifiedFieldNames=True):
            arcpy.env.overwriteOutput = True
            csvfile = "{}\\temp.csv".format(path_temp)
            arcpy.conversion.ExportTable(in_table=filename,
                                         out_table=csvfile,
                                         where_clause="",
                                         use_field_alias_as_name="NOT_USE_ALIAS",
                                         field_mapping="",
                                         sort_field=[])
        df_output = pd.read_csv(csvfile)
    return df_output

In [26]:
ReadFeatureLayer(filename, path_gdb, path_temp)

,OID_,ObjectID,FrameIndex,Coord_X,Coord_Y,Coord_Z,Speed_X,Speed_Y,Speed,Longitude,...,Width,Area,Dis,Max_Length,Class,MoveAngle,DateTime,AdjSpeed,MedianSpeed,Acceleration
0,1,7.0,2.0,-50.429108,-64.546315,0.0,0.000000,0.000000,0.000000,-119.809674,...,2.182359,0.312239,82.080000,2.438856,1,NaN,2021-12-19 09:30:00.200,NaN,NaN,NaN
1,2,7.0,3.0,-49.564950,-63.817335,0.0,4.019341,3.390606,5.258452,-119.809671,...,2.182359,0.418657,80.758000,2.438856,1,NaN,2021-12-19 09:30:00.300,25.290108,25.290108,NaN
2,3,7.0,4.0,-49.013924,-62.484297,0.0,4.696644,7.906111,9.195926,-119.809664,...,2.182359,0.046361,79.312000,2.438856,1,NaN,2021-12-19 09:30:00.400,32.266394,28.778251,NaN
3,4,7.0,5.0,-48.544260,-61.693686,0.0,4.696644,7.906111,9.195926,-119.809660,...,2.182359,0.046361,79.312000,2.438856,1,NaN,2021-12-19 09:30:00.500,20.570734,25.290108,NaN
4,5,7.0,6.0,-47.596023,-60.440993,0.0,6.447360,9.596505,11.561201,-119.809655,...,2.182359,0.156016,76.728800,2.438856,1,NaN,2021-12-19 09:30:00.600,35.144844,28.778251,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174706,174707,11562.0,17832.0,-28.579268,11.069112,0.0,0.056013,-5.797878,5.798149,-119.809122,...,2.548854,5.449284,30.293034,3.501326,1,256.932651,2021-12-19 09:59:43.200,25.630982,9.952454,-13.606850
174707,174708,11562.0,17833.0,-29.145902,10.223180,0.0,-1.308714,-6.432608,6.564387,-119.809124,...,2.548854,8.990108,30.543858,3.501326,1,267.439694,2021-12-19 09:59:43.300,22.775922,9.952454,-4.678539
174708,174709,11562.0,17834.0,-29.708009,9.508288,0.0,-2.337068,-6.603425,7.004791,-119.809122,...,2.548854,13.342719,30.650906,3.501326,1,300.691015,2021-12-19 09:59:43.400,20.343057,14.234118,6.279775
174709,174710,11562.0,17835.0,-30.584570,8.597442,0.0,-3.869985,-7.200760,8.174823,-119.809124,...,2.548854,5.482132,30.974867,3.501326,1,349.587226,2021-12-19 09:59:43.500,28.277596,18.607081,12.693454


In [34]:
def AddGlobalID(df,mapName,IDName):
    list_global_id = []
    for i in range(len(df)):
        dateInfo = mapName[3:15]
        localID = int(df.at[df.index[i], IDName])
        list_global_id.append(dateInfo + '-' + str(localID))
    df['GlobalID'] = list_global_id
    return df

In [43]:
def TaskCounter(counter, total_number):
    counter = counter + 1
    if counter == 1:
        print('Finish {} job'.format(counter) + ', {} remaining.'.format(total_number - counter))
    elif counter != total_number:
        print('Finish {} jobs'.format(counter) + ', {} remaining.'.format(total_number-counter))
    return counter

In [50]:
df_all = pd.DataFrame()

In [51]:
with arcpy.EnvManager(scratchWorkspace=path_gdb, workspace=path_gdb):
    dataset = arcpy.ListFeatureClasses(feature_type='Point', wild_card=wildcard)
    counter_task = 0
    for map in dataset:
        df = AddGlobalID(ReadFeatureLayer(map, path_gdb, path_temp),map,'ObjectID' )
        df.drop(['DateTime', 'ObjectID'], axis=1, inplace=True)
        columns_to_convert = ['FrameIndex', 'Point_Cnt']  # replace with your column names
        for col in columns_to_convert:
            df[col] = df[col].astype(int)
        df_all = pd.concat([df,df_all],ignore_index=True)
        counter_task = TaskCounter(counter_task, len(dataset))

Finish 1 job, 47 remaining.
Finish 2 jobs, 46 remaining.
Finish 3 jobs, 45 remaining.
Finish 4 jobs, 44 remaining.
Finish 5 jobs, 43 remaining.
Finish 6 jobs, 42 remaining.
Finish 7 jobs, 41 remaining.
Finish 8 jobs, 40 remaining.
Finish 9 jobs, 39 remaining.
Finish 10 jobs, 38 remaining.
Finish 11 jobs, 37 remaining.
Finish 12 jobs, 36 remaining.
Finish 13 jobs, 35 remaining.
Finish 14 jobs, 34 remaining.
Finish 15 jobs, 33 remaining.
Finish 16 jobs, 32 remaining.
Finish 17 jobs, 31 remaining.
Finish 18 jobs, 30 remaining.
Finish 19 jobs, 29 remaining.
Finish 20 jobs, 28 remaining.
Finish 21 jobs, 27 remaining.
Finish 22 jobs, 26 remaining.
Finish 23 jobs, 25 remaining.
Finish 24 jobs, 24 remaining.
Finish 25 jobs, 23 remaining.
Finish 26 jobs, 22 remaining.
Finish 27 jobs, 21 remaining.
Finish 28 jobs, 20 remaining.
Finish 29 jobs, 19 remaining.
Finish 30 jobs, 18 remaining.
Finish 31 jobs, 17 remaining.
Finish 32 jobs, 16 remaining.
Finish 33 jobs, 15 remaining.
Finish 34 jobs, 14 r

In [53]:
df_all

,OID_,FrameIndex,Coord_X,Coord_Y,Coord_Z,Speed_X,Speed_Y,Speed,Longitude,Latitude,...,Width,Area,Dis,Max_Length,Class,MoveAngle,AdjSpeed,MedianSpeed,Acceleration,GlobalID
0,1,2,-50.429108,-64.546315,0.0,0.000000,0.000000,0.000000,-119.809674,39.519009,...,2.182359,0.312239,82.080000,2.438856,1,NaN,NaN,NaN,NaN,202112190930-7
1,2,3,-49.564950,-63.817335,0.0,4.019341,3.390606,5.258452,-119.809671,39.519000,...,2.182359,0.418657,80.758000,2.438856,1,NaN,25.290108,25.290108,NaN,202112190930-7
2,3,4,-49.013924,-62.484297,0.0,4.696644,7.906111,9.195926,-119.809664,39.518987,...,2.182359,0.046361,79.312000,2.438856,1,NaN,32.266394,28.778251,NaN,202112190930-7
3,4,5,-48.544260,-61.693686,0.0,4.696644,7.906111,9.195926,-119.809660,39.518979,...,2.182359,0.046361,79.312000,2.438856,1,NaN,20.570734,25.290108,NaN,202112190930-7
4,5,6,-47.596023,-60.440993,0.0,6.447360,9.596505,11.561201,-119.809655,39.518965,...,2.182359,0.156016,76.728800,2.438856,1,NaN,35.144844,28.778251,NaN,202112190930-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343795,72320,17832,-7.420439,9.800035,0.0,-6.056178,-9.620730,11.368189,-119.809281,39.518288,...,2.794487,11.179612,12.543144,4.491904,1,338.618289,25.386272,25.270079,2.664082,202112190000-6713
6343796,72321,17833,-8.006769,8.826333,0.0,-6.010187,-9.648459,11.367282,-119.809285,39.518298,...,2.794487,11.691995,12.019200,4.491904,1,339.492302,25.425246,25.270079,2.664082,202112190000-6713
6343797,72322,17834,-8.478235,7.742124,0.0,-5.701279,-9.933071,11.452969,-119.809297,39.518308,...,2.794487,8.812228,11.572815,4.491904,1,333.904976,26.446933,25.405759,2.863078,202112190000-6713
6343798,72323,17835,-8.833774,6.602680,0.0,-5.189609,-10.281522,11.517019,-119.809299,39.518316,...,2.794487,7.887994,11.244717,4.491904,1,335.059181,26.700662,25.405759,2.439895,202112190000-6713


In [54]:
df_all.to_csv('{}\Lawrence1219.csv'.format(path_temp))